In [23]:
import osmnx as ox

In [24]:
# Define the point's latitude and longitude
lat = 46.94456
lon = 7.41435


In [148]:
# Check if rail is closer than dist to the point (lat, lon)
def isRailCloserThan(lat, lon, dist):
    # Get the graph and nearest edge using OSMnx
    try:
        # Here, we consider only main and branch lines (not "parkings" etc.), that we cannot do in the features_from_point version
        G = ox.graph_from_point((lat, lon), custom_filter = '["railway"~"rail"]["usage"~"main|branch"]', dist=dist, simplify=False, retain_all=True)
        #ox.plot_graph(G)
        #nearest_edge = ox.distance.nearest_edges(G, lon, lat, return_dist=True)
        # Multiply the distance in degrees by 111,139 to get the distance in meters (https://sciencing.com/convert-latitude-longtitude-feet-2724.html)
        return True
    except:
        return False

In [149]:
isRailCloserThan(lat, lon, 1000)

True

In [150]:
# Check if rails exist closer than dist to the point (lat, lon), QUICK VERSION!
def is_rail_closer_than(lat, lon, dist):
    try:
        res = ox.features_from_point((lat, lon), dist=dist, tags = {"railway":"rail", })
        return len(res) > 0
    except:
        return False

In [151]:
%%timeit
isRailCloserThan(lat, lon, 8)

39.7 ms ± 1.13 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [152]:
%%timeit
# Check if there are rails closer than 8 meters to the point where the user is ( this can be used to know if a person is in a train or not)
is_rail_closer_than(lat, lon, 8)

6.33 ms ± 654 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [4]:
import overpy
import csv

In [5]:
api = overpy.Overpass()

In [32]:
full_query = """
// [out:csv(::type, "railway", "route", "natural")] // => CSV output not supported by python overpass wrapper
[out:json]
[timeout:25];
// gather results
(
	// Close to a train track?
  	way["railway"="rail"]["usage"="main"](around: 100, 46.996296, 6.933764);
  	way["railway"="rail"]["usage"="branch"](around: 100, 46.996296, 6.933764);
  
  	// Close to tram rails?
  	way["railway"="tram"](around: 100, 46.996296, 6.933764);
  	
  	// Close to bus line?
  	rel["route"="bus"](around: 100, 46.996296, 6.933764);
  	rel["route"="trolleybus"](around: 100, 46.996296, 6.933764);
  
	// Are we on a lake? (Double set negation because area.a["natural" != "water"] does not work... => (area.a; - area.a["natural"];) )
	(is_in(46.996709, 7.0)->.a; - (area.a; - area.a["natural"];););
);
// Export results
out body;
"""

In [33]:
result = api.query(full_query)

In [34]:
for area in result.areas:
    print("Area: ", area.tags)

Area:  {'ele': '429', 'intermittent': 'no', 'name': 'Lac de Neuchâtel', 'name:ca': 'Lac de Neuchâtel', 'name:cs': 'Neuchâtelské jezero', 'name:da': 'Neuchâtelsøen', 'name:de': 'Neuenburgersee', 'name:en': 'Lake Neuchâtel', 'name:fr': 'Lac de Neuchâtel', 'name:hr': 'Neušatelsko jezero', 'name:lt': 'Nešatelio ežeras', 'name:ru': 'Невшательское озеро', 'name:sk': 'Neuchâtelské jazero', 'name:sl': 'Neuchâtelsko jezero', 'natural': 'water', 'salt': 'no', 'tidal': 'no', 'type': 'multipolygon', 'water': 'lake', 'wikidata': 'Q14375', 'wikipedia': 'fr:Lac de Neuchâtel'}


In [35]:
for way in result.ways:
    print("Way: ", way.tags)

Way:  {'electrified': 'contact_line', 'frequency': '16.7', 'gauge': '1435', 'maxspeed': '95', 'operator': 'SBB', 'passenger_lines': '5', 'railway': 'rail', 'railway:etcs': '1', 'tracks': '1', 'usage': 'main', 'voltage': '15000'}
Way:  {'electrified': 'contact_line', 'frequency': '16.7', 'gauge': '1435', 'maxspeed': '95', 'operator': 'SBB', 'passenger_lines': '5', 'railway': 'rail', 'railway:etcs': '1', 'railway:track_ref': '3', 'tracks': '1', 'usage': 'main', 'voltage': '15000'}
Way:  {'electrified': 'contact_line', 'frequency': '16.7', 'gauge': '1435', 'maxspeed': '95', 'operator': 'SBB', 'passenger_lines': '5', 'railway': 'rail', 'railway:etcs': '1', 'railway:track_ref': '4', 'tracks': '1', 'usage': 'main', 'voltage': '15000'}
Way:  {'electrified': 'contact_line', 'frequency': '16.7', 'gauge': '1435', 'maxspeed': '95', 'operator': 'SBB', 'passenger_lines': '5', 'railway': 'rail', 'railway:etcs': '1', 'tracks': '1', 'usage': 'main', 'voltage': '15000'}
